In [ ]:
!pip install transformers
!pip install -q torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files,drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import sys
sys.path.append('/gdrive/MyDrive/finn-project/be-ai-model')

from data_processing import get_csv_data, merge_data
from sentiment_model import analyze_sentiment_with_progress, add_integer_column, sums_sentiment_score_for_7_days
from lstm_model import get_train_scale_data, get_test_scale_data, create_sequences, compile_model, train_model, predict_prices
from final_result import compare_prices_with_graph

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Device set to use cuda:0


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
import pandas as pd

news_train_df = get_csv_data("/gdrive/MyDrive/finn-project/tsla_news_train.csv")
news_test_df = get_csv_data("/gdrive/MyDrive/finn-project/tsla_news_test.csv")

news_train_df = analyze_sentiment_with_progress(news_train_df)
news_test_df = analyze_sentiment_with_progress(news_test_df)

감성 분석 진행중:   0%|          | 0/391 [00:00<?, ?batch/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


감성 분석 진행중:   0%|          | 0/224 [00:00<?, ?batch/s]

In [ ]:
stock_train_df = get_csv_data("/gdrive/MyDrive/finn-project/tsla_prices_train.csv")
stock_test_df = get_csv_data("/gdrive/MyDrive/finn-project/tsla_prices_test.csv")

stock_train_df['date'] = pd.to_datetime(stock_train_df['date']).dt.strftime('%Y-%m-%d')
stock_test_df['date'] = pd.to_datetime(stock_test_df['date']).dt.strftime('%Y-%m-%d')

In [ ]:
merged_train = merge_data(news_train_df, stock_train_df)
merged_test = merge_data(news_test_df, stock_test_df)

merged_train['sentiment_influence'] = 0.0
merged_test['sentiment_influence'] = 0.0

merged_train = add_integer_column(merged_train)
merged_test = add_integer_column(merged_test)

sums_sentiment_score_for_7_days(merged_train)
sums_sentiment_score_for_7_days(merged_test)

  0%|          | 0/12482 [00:00<?, ?it/s]

  0%|          | 0/7141 [00:00<?, ?it/s]

In [ ]:
train_scaled = get_train_scale_data(merged_train)
test_scaled = get_test_scale_data(merged_test)

X_train, y_train = create_sequences(train_scaled)
X_test, y_test = create_sequences(test_scaled)

In [ ]:
from tensorflow.keras.models import load_model

def get_existing_model(path):
  load_path = '/content/drive/MyDrive/models/my_lstm_model.h5'
  return load_model(load_path)

In [ ]:
model = compile_model(X_train)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        18,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,241 (71.25 KB)

 Trainable params: 18,241 (71.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model = train_model(model, X_train, y_train, X_test, y_test)

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node sequential_1/lstm_1/CudnnRNNV3 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dis

In [ ]:
y_pred = predict_prices(model, X_test)

In [ ]:
compare_prices_with_graph(y_test, y_pred)

In [ ]:
save_path = '/gdrive/MyDrive/finn-project/Finn'
model.save(save_path)